In [ ]:

!pip install langchain langchain-community langchain-openai
!pip install chromadb
!pip install pypdf
!pip install sentence-transformers
!pip install openai

In [26]:

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import tempfile

In [ ]:
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key:")

In [28]:
# Cell 4: Document Processing Functions
def load_pdf(pdf_path):
    """
    Load PDF and extract text
    """
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from PDF")
    return documents

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Split documents into smaller chunks for better retrieval
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    splits = text_splitter.split_documents(documents)
    print(f"Split into {len(splits)} chunks")
    return splits

In [29]:
# Initialize Embeddings
def setup_embeddings():
    """
    Initialize embedding model - using free HuggingFace model
    """
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    print("Embeddings model loaded successfully!")
    return embeddings

# Initialize embeddings
embeddings = setup_embeddings()

Embeddings model loaded successfully!


In [30]:
# Vector Database Functions
def create_vector_store(documents, embeddings):
    """
    Create vector database from document chunks
    """
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory="./chroma_db"  # Save to disk
    )
    print(f"Created vector store with {len(documents)} documents")
    return vectorstore

def load_existing_vector_store(embeddings):
    """
    Load existing vector database if it exists
    """
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )
    return vectorstore

In [31]:
#  RAG Chain Setup
def create_rag_chain(vectorstore):
    """
    Create the conversational RAG chain
    """
    # Create retriever from vector store
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}  # Retrieve top 3 most similar chunks
    )

    # Initialize LLM
    llm = OpenAI(temperature=0.7)

    # Create memory for conversation history
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

    # Create conversational chain
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        verbose=True
    )

    print("RAG chain created successfully!")
    return qa_chain

In [32]:
# PDF Upload and Processing Function
def process_pdf_file(uploaded_file):
    """
    Process uploaded PDF file and create vector store
    """
    try:
        # Save uploaded file temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            tmp_file.write(uploaded_file.read())
            tmp_file_path = tmp_file.name

        print("Step 1: Loading PDF...")
        documents = load_pdf(tmp_file_path)

        print("Step 2: Splitting documents...")
        splits = split_documents(documents)

        print("Step 3: Creating vector store...")
        vectorstore = create_vector_store(splits, embeddings)

        print("Step 4: Creating RAG chain...")
        qa_chain = create_rag_chain(vectorstore)

        # Clean up temporary file
        os.unlink(tmp_file_path)

        print("PDF processed successfully! You can now ask questions.")
        return qa_chain, vectorstore

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return None, None

In [33]:
# Chat Interface Function
def chat_with_pdf(qa_chain, question):
    """
    Ask questions about the PDF content
    """
    try:
        print(f"Question: {question}")
        print("Searching relevant content...")

        # Get response from RAG chain
        response = qa_chain({"question": question})

        print(f"Answer: {response['answer']}")

        # Show source documents if available
        if 'source_documents' in response and response['source_documents']:
            print("\n Sources used:")
            for i, doc in enumerate(response['source_documents']):
                print(f"   Source {i+1}: Page {doc.metadata.get('page', 'Unknown')}")
                print(f"   Content preview: {doc.page_content[:200]}...")
                print()

        return response

    except Exception as e:
        print(f"Error during chat: {str(e)}")
        return None

In [34]:
# File Upload Helper for Colab
from google.colab import files

def upload_and_process_pdf():
    """
    Upload PDF file in Colab and process it
    """
    print("Please upload your PDF file...")
    uploaded = files.upload()

    if uploaded:
        # Get the first uploaded file
        filename = list(uploaded.keys())[0]
        file_content = uploaded[filename]

        print(f"Processing {filename}...")

        # Create a file-like object
        import io
        file_obj = io.BytesIO(file_content)

        # Process the PDF
        qa_chain, vectorstore = process_pdf_file(file_obj)

        return qa_chain, vectorstore
    else:
        print("No file uploaded")
        return None, None

In [35]:
# Initialize the System
print("RAG PDF Chatbot System Ready!")
print("=" * 50)

# Global variables to store our system
qa_chain = None
vectorstore = None

print(" System initialized. Ready to process PDF!")

RAG PDF Chatbot System Ready!
 System initialized. Ready to process PDF!


In [ ]:
# Upload and Process PDF
print("Upload and Process Your PDF")
print("=" * 40)

# Upload and process PDF
qa_chain, vectorstore = upload_and_process_pdf()

if qa_chain:
    print("Success! Your PDF is now ready for questions.")
    print("You can now ask questions about the content in the next cell.")
else:
    print("Failed to process PDF. Please try again.")

In [38]:
# Interactive Chat Interface
def start_chat():
    """
    Start interactive chat session
    """
    if qa_chain is None:
        print("Please upload and process a PDF first!")
        return

    print("💬 Chat with your PDF - Type 'quit' to exit")
    print("=" * 45)

    while True:
        question = input("\n Your question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break

        if not question:
            print("Please enter a question.")
            continue

        # Get response
        response = chat_with_pdf(qa_chain, question)

        if response:
            print("-" * 50)

# Start the chat
start_chat()

Please upload and process a PDF first!


In [ ]:
##Token limit in api call of OpenAI

In [ ]:
##FreeModel- Using local model

In [ ]:
!pip install langchain-huggingface

In [39]:
# Setup Free Local LLM
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

def setup_free_llm():
    """
    Setup free local language model
    """
    print(" Loading free local model (this may take a few minutes first time)...")

    # Use a smaller, efficient model that works well in Colab
    model_name = "microsoft/DialoGPT-medium"

    # Alternative models you can try:
    # "google/flan-t5-base" - Good for Q&A
    # "microsoft/DialoGPT-small" - Smaller, faster

    try:
        # Create pipeline
        pipe = pipeline(
            "text-generation",
            model=model_name,
            tokenizer=model_name,
            max_length=512,
            temperature=0.7,
            do_sample=True,
            device=0 if torch.cuda.is_available() else -1  # Use GPU if available
        )

        # Wrap in LangChain
        llm = HuggingFacePipeline(pipeline=pipe)
        print("Free local model loaded successfully!")
        return llm

    except Exception as e:
        print(f"Error loading model: {e}")
        print("Trying alternative model...")

        # Fallback to even smaller model
        pipe = pipeline(
            "text-generation",
            model="gpt2",
            max_length=256,
            temperature=0.7,
            do_sample=True
        )
        llm = HuggingFacePipeline(pipeline=pipe)
        print("Fallback model loaded!")
        return llm

# Load the free model
free_llm = setup_free_llm()

 Loading free local model (this may take a few minutes first time)...


Device set to use cpu


Free local model loaded successfully!


In [ ]:
# Check Current Variables
print("🔍 Checking current variables...")
print(f"vectorstore exists: {'vectorstore' in globals() and vectorstore is not None}")
print(f"qa_chain exists: {'qa_chain' in globals() and qa_chain is not None}")
print(f"embeddings exists: {'embeddings' in globals() and embeddings is not None}")
print(f"free_llm exists: {'free_llm' in globals() and free_llm is not None}")

# Check if we have any Chroma database saved
import os
chroma_exists = os.path.exists('./chroma_db')
print(f"Chroma database exists on disk: {chroma_exists}")

In [41]:
# Modified RAG Chain with Free LLM
def create_free_rag_chain(vectorstore):
    """
    Create RAG chain using free local model
    """
    # Create retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 2}  # Reduced to 2 for smaller context
    )

    # Use our free LLM
    llm = free_llm

    # Create memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

    # Create chain
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        verbose=True
    )

    print("✅ Free RAG chain created!")
    return qa_chain

# If you already have a vectorstore from before, recreate the chain
if vectorstore is not None:
    print("🔄 Recreating RAG chain with free model...")
    qa_chain = create_free_rag_chain(vectorstore)
    print("✅ Ready to chat with free model!")
else:
    print("📤 Please upload a PDF first using the upload function")

📤 Please upload a PDF first using the upload function


In [ ]:
# Created Simpler QA Function
def simple_qa(question):
    """
    Simple QA function that works with free model
    """
    print(f"🤔 Question: {question}")

    # Get relevant chunks
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    docs = retriever.get_relevant_documents(question)

    if not docs:
        print("❌ No relevant documents found")
        return

    # Combine context
    context = "\n".join([doc.page_content[:300] for doc in docs])

    # Create simple prompt
    prompt = f"""Based on the following context, answer the question:

Context: {context}

Question: {question}

Answer:"""

    print("🤖 Generating answer...")
    try:
        answer = free_llm.predict(prompt)
        print(f"✅ Answer: {answer}")

        # Show sources
        print("\n📄 Sources:")
        for i, doc in enumerate(docs):
            print(f"Source {i+1}: {doc.page_content[:100]}...")

        return answer
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

# Test the simple function
simple_qa("What is this document about?")

In [ ]:
# Interactive Chat with Simple Function
def simple_chat():
    """
    Simple chat interface using our working function
    """
    print("💬 Simple PDF Chat (type 'quit' to exit)")
    print("=" * 40)

    while True:
        question = input("\n🤔 Your question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break

        if not question:
            continue

        simple_qa(question)
        print("-" * 40)

# Start simple chat
simple_chat()